In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
train_data = pd.read_csv('train-data.csv')
test_data = pd.read_csv('test-data.csv')

In [3]:
train_data = train_data.drop(['Unnamed: 0', 'New_Price'], axis=1)
test_data = test_data.drop(['Unnamed: 0', 'New_Price'], axis=1)

In [4]:
train_data['Flag'] = 1
test_data['Flag'] = 0

In [5]:
combined_data = pd.concat([train_data, test_data])

In [6]:
combined_data['Mileage'] = combined_data['Mileage'].str.extract('(\d+\.\d+|\d+)').astype(float)
combined_data['Engine'] = combined_data['Engine'].str.extract('(\d+)').astype(float).fillna(0).astype(int)
combined_data['Power'] = combined_data['Power'].str.extract('(\d+\.\d+|\d+)').astype(float)

In [7]:
label_encoder = LabelEncoder()
categorical_cols = ['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type']
for col in categorical_cols:
    combined_data[col] = label_encoder.fit_transform(combined_data[col])

In [8]:
onehot_encoder = OneHotEncoder(sparse=False)
encoded_data = onehot_encoder.fit_transform(combined_data)

In [9]:
X_train = encoded_data[combined_data['Flag'] == 1, :-1]
y_train = encoded_data[combined_data['Flag'] == 1, -1]
X_test = encoded_data[combined_data['Flag'] == 0, :-1]

In [10]:
X_train = np.nan_to_num(X_train)
y_train = np.nan_to_num(y_train)
X_test = np.nan_to_num(X_test)

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='relu'))

In [13]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [14]:
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32)

Epoch 1/10
189/189 [==============================] - 2s 13ms/step - loss: 0.3003
Epoch 2/10
189/189 [==============================] - 2s 10ms/step - loss: 0.3816
Epoch 3/10
189/189 [==============================] - 2s 10ms/step - loss: 0.4010
Epoch 4/10
189/189 [==============================] - 2s 10ms/step - loss: 0.2077
Epoch 5/10
189/189 [==============================] - 2s 10ms/step - loss: 0.0987
Epoch 6/10
189/189 [==============================] - 2s 10ms/step - loss: 0.0430
Epoch 7/10
189/189 [==============================] - 2s 10ms/step - loss: 0.0186
Epoch 8/10
189/189 [==============================] - 2s 10ms/step - loss: 0.0120
Epoch 9/10
189/189 [==============================] - 2s 10ms/step - loss: 0.0054
Epoch 10/10
189/189 [==============================] - 2s 10ms/step - loss: 0.0036


In [15]:
predictions = model.predict(X_test_scaled)


print(predictions)

[[0.7270334 ]
 [0.97732383]
 [1.0129025 ]
 ...
 [0.94560343]
 [0.9402496 ]
 [0.9921563 ]]
